In [1]:
import os

os.chdir('../')
%pwd

'/home/eman/Desktop/DataScience_proj'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    rootdir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: Path
    alpha: float
    l1_ratio: float
    random_state: int
    TARGET_COLUMN:str

In [3]:
from src.Datascience_proj.utils.common import read_yaml, create_directories, save_bin
from src.Datascience_proj.constants import *

In [4]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
        ## read the file and assign to variable
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        ##creating our artifacts directory
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema= self.schema.TARGET_COLUMN
        params2= self.params.data_params

        ## creating a new directory
        create_directories([config.root_dir])

        model_trainer = ModelTrainerConfig(
            rootdir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            random_state = params2.random_state,
            TARGET_COLUMN = schema.name
        )

        return model_trainer

In [5]:
import pandas as pd
from src.Datascience_proj import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config= config

    def train_model(self):
        
        ## importing the training and test dataset
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        ## splitting independent and target features
        #train
        train_X = train_data.drop([self.config.TARGET_COLUMN], axis=1)
        train_y = train_data[[self.config.TARGET_COLUMN]]

        #test
        test_X = test_data.drop([self.config.TARGET_COLUMN],axis=1)
        test_y = test_data[[self.config.TARGET_COLUMN]]

        ## initializing ml model
        lr = ElasticNet(alpha = self.config.alpha, l1_ratio= self.config.l1_ratio, 
                        random_state = self.config.random_state)

        ## training the model
        lr.fit(train_X, train_y)

        ## saving the model
        joblib.dump(lr, self.config.model_name)


In [11]:
config= ConfigurationManager()
config_model = config.get_model_trainer_config()
model_trainer = ModelTrainer(config_model)
model_trainer.train_model()

[2025-07-16 15:59:50,976: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-16 15:59:50,984: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-16 15:59:50,991: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-16 15:59:50,993: INFO: common: created directory at: artifacts]
[2025-07-16 15:59:51,003: INFO: common: created directory at: artifacts/model_trainer]
